# Dependencies

In [30]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

# Model

In [31]:
class Alexnet(nn.Module):
    def __init__(self, num_classes=45):
        super().__init__()

        self.net = nn.Sequential(
            self.conv_blocks(3, 96, 11, 4, max_pool=True, local_normalization=True),  # b X 96 X 55 X 55 -> maxpool: b X 96 X 27 X 27
            self.conv_blocks(96, 256, 5, padding=2, max_pool=True, local_normalization=True),  # b X 256 X 27 X 27 -> maxpool: b X 256 X 13 X 13
            self.conv_blocks(256, 384, 3, padding=1),  # b X 384 X 13 X 13
            self.conv_blocks(384, 384, 3, padding=1),  # b X 384 X 13 X 13
            self.conv_blocks(384, 256, 3, padding=1, max_pool=True),  # b X 256 X 13 X 13 -> maxpool: b X 256 x 6 X 6
        )
        self.fc = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.net(x)
        x = x.view(-1, 256 * 6 * 6)
        x = self.fc(x)
        return x

    def conv_blocks(self, in_channels, out_channels, kernel_size, stride=1, padding=0, max_pool=False, local_normalization=False):
        layers = []
        layers += [nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)]
        layers += [nn.ReLU()]
        if local_normalization:
            layers += [nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2)]
        if max_pool:
            layers += [nn.MaxPool2d((3, 3), stride=2)]
        return nn.Sequential(*layers)

    def init_bias(self):
        for layer in self.net:
            if isinstance(layer, nn.Conv2d):
                nn.init.normal_(layer.weight, mean=0, std=0.01)
                nn.init.constant_(layer.bias, 0)
        nn.init.constant_(self.net[0][0].bias, 1)
        nn.init.constant_(self.net[3][0].bias, 1)
        nn.init.constant_(self.net[4][0].bias, 1)


# HyperParameters

In [44]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
IMAGE_SIZE = 227
BATCH_SIZE = 64
LR = 0.0001
EPOCHS = 90
PATH = "model_checkpoint.pth"

# Dataset

In [12]:
# prompt: /content/drive/MyDrive/data/Mammals_Images.zip extract

!unzip /content/drive/MyDrive/data/Mammals_Images.zip -d /content

Archive:  /content/drive/MyDrive/data/Mammals_Images.zip
replace /content/mammals/african_elephant/african_elephant-0001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [45]:
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    #transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
])

dataset = datasets.ImageFolder(root='/content/mammals', transform = transform)
dataloader = DataLoader(dataset, batch_size = BATCH_SIZE, shuffle=True, num_workers = 2, drop_last=True, pin_memory = True)

# Model Initialization



In [46]:
model = Alexnet().to(DEVICE)
model.init_bias()
optimizer = torch.optim.Adam(model.parameters(), lr = LR)
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

# Training

In [47]:
best_accuracy = 0.0

for epoch in range(EPOCHS):
    lr_scheduler.step()
    epoch_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for image, label in tqdm(dataloader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)

        optimizer.zero_grad()
        score = model(image)
        loss = criterion(score, label)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        _, preds = torch.max(score, 1)
        correct_predictions += torch.sum(preds == label).item()
        total_samples += label.size(0)
    epoch_accuracy = correct_predictions / total_samples
    epoch_loss /= len(dataloader)

    print('Epoch: {} \tLoss: {:.4f} \tAcc: {:.4f}'.format(epoch + 1, epoch_loss, epoch_accuracy))

    # Save the model if it has the best accuracy so far
    if epoch_accuracy > best_accuracy:
        best_accuracy = epoch_accuracy
        torch.save({'epoch': epoch + 1,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'best_accuracy': best_accuracy}, PATH)


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
  0%|          | 0/214 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 214/214 [00:42<00:00,  5.07it/s]


Epoch: 1 	Loss: 3.8096 	Acc: 0.0223


100%|██████████| 214/214 [00:41<00:00,  5.11it/s]


Epoch: 2 	Loss: 3.7474 	Acc: 0.0352


100%|██████████| 214/214 [00:41<00:00,  5.10it/s]


...

Epoch: 45 	Loss: 0.0335 	Acc: 0.9899


100%|██████████| 214/214 [00:41<00:00,  5.12it/s]


Epoch: 46 	Loss: 0.0374 	Acc: 0.9897


 21%|██        | 44/214 [00:08<00:33,  5.02it/s]


KeyboardInterrupt: 

In [ ]:
# Load model and optimizer state_dicts
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
